In [1]:
!pip install geocoder

In [2]:

import pandas as pd
import numpy as np
import geocoder
print("Imported!")

Imported!


In [3]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df_list = pd.read_html(url)[0]

In [5]:
# clean dataframe 
df_list = df_list[df_list.Borough!='Not assigned']
df_list = df_list[df_list.Borough!=0]
df_list.reset_index(drop = True, inplace = True)

i = 0
for i in range(0,df_list.shape[0]):
    if df_list.iloc[i][2] == 'Not assigned':
        df_list.iloc[i][2] = df_list.iloc[i][1]
        i = i+1

In [25]:

df = df_list.groupby(['Postal Code','Borough'])['Neighbourhood'].apply(', '.join).reset_index()
df

Postal Code      Borough  \
0           M1B  Scarborough   
1           M1C  Scarborough   
2           M1E  Scarborough   
3           M1G  Scarborough   
4           M1H  Scarborough   
..          ...          ...   
98          M9N         York   
99          M9P    Etobicoke   
100         M9R    Etobicoke   
101         M9V    Etobicoke   
102         M9W    Etobicoke   

                                         Neighbourhood  
0                                       Malvern, Rouge  
1               Rouge Hill, Port Union, Highland Creek  
2                    Guildwood, Morningside, West Hill  
3                                               Woburn  
4                                            Cedarbrae  
..                                                 ...  
98                                              Weston  
99                                           Westmount  
100  Kingsview Village, St. Phillips, Martin Grove ...  
101  South Steeles, Silverstone, Humbergate, Jamest...  
102                Northwest, West Humber - Clairville  

[103 rows x 3 columns]

In [26]:
def get_latilong(postal_code):
    lati_long_coords = None
    while(lati_long_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
        lati_long_coords = g.latlng
    return lati_long_coords
    
get_latilong('M4G')

[43.709020000000066, -79.36348999999996]

In [27]:

# Retrieving Postal Code Co-ordinates
postal_codes = df['Postal Code']    
coords = [ get_latilong(postal_code) for postal_code in postal_codes.tolist() ]

In [30]:

# Adding Columns Latitude & Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])
df['Latitude'] = df_coords['Latitude']
df['Longitude'] = df_coords['Longitude']

In [31]:
df[postal_codes == 'M5G']

Postal Code           Borough       Neighbourhood  Latitude  Longitude
57         M5G  Downtown Toronto  Central Bay Street  43.65609  -79.38493

In [32]:

df.head(15)

Postal Code      Borough  \
0          M1B  Scarborough   
1          M1C  Scarborough   
2          M1E  Scarborough   
3          M1G  Scarborough   
4          M1H  Scarborough   
5          M1J  Scarborough   
6          M1K  Scarborough   
7          M1L  Scarborough   
8          M1M  Scarborough   
9          M1N  Scarborough   
10         M1P  Scarborough   
11         M1R  Scarborough   
12         M1S  Scarborough   
13         M1T  Scarborough   
14         M1V  Scarborough   

                                        Neighbourhood  Latitude  Longitude  
0                                      Malvern, Rouge  43.81139  -79.19662  
1              Rouge Hill, Port Union, Highland Creek  43.78574  -79.15875  
2                   Guildwood, Morningside, West Hill  43.76575  -79.17470  
3                                              Woburn  43.76812  -79.21761  
4                                           Cedarbrae  43.76944  -79.23892  
5                                 Scarborough Village  43.74446  -79.23117  
6         Kennedy Park, Ionview, East Birchmount Park  43.72582  -79.26461  
7                     Golden Mile, Clairlea, Oakridge  43.71289  -79.28506  
8     Cliffside, Cliffcrest, Scarborough Village West  43.72360  -79.23496  
9                         Birch Cliff, Cliffside West  43.69510  -79.26466  
10  Dorset Park, Wexford Heights, Scarborough Town...  43.75998  -79.26940  
11                                  Wexford, Maryvale  43.75075  -79.30054  
12                                          Agincourt  43.79452  -79.26708  
13            Clarks Corners, Tam O'Shanter, Sullivan  43.78491  -79.29722  
14  Milliken, Agincourt North, Steeles East, L'Amo...  43.81781  -79.28044